In [25]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 10.7 ms (started: 2021-08-27 15:43:58 +08:00)


In [61]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #3 Aug 27, 2021'

#git push origin master
git push

origin	git@github.com:ustchope/keras_examples.git (fetch)
origin	git@github.com:ustchope/keras_examples.git (push)
[main 0d17081] 更新 #3 Aug 27, 2021
 1 file changed, 1158 insertions(+), 125 deletions(-)


To git@github.com:ustchope/keras_examples.git
   5cf5b12..0d17081  main -> main


time: 4.28 s (started: 2021-08-27 17:51:50 +08:00)


In [29]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 997 µs (started: 2021-08-27 15:44:25 +08:00)


In [4]:
tf.__version__

'2.6.0'

time: 3.21 ms (started: 2021-08-27 14:11:07 +08:00)


# 简单的自定义层示例：Antirectifier

## 介绍
此示例展示了如何使用 Antirectifier 层（最初在 2016 年 1 月作为 Keras 示例脚本提出）创建自定义层，这是 ReLU 的替代方案。 它不是将输入的负部分归零，而是将负部分和正部分分开，并返回两者绝对值的串联。 这避免了信息丢失，代价是维度增加。 为了修复维数增加，我们将特征线性组合回原始大小的空间。

## 设置

In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

time: 530 µs (started: 2021-08-27 15:44:32 +08:00)


## 抗整流层

In [6]:
class Antirectifier(layers.Layer):
    def __init__(self, initializer="he_normal", **kwargs):
        super(Antirectifier, self).__init__(**kwargs)
        self.initializer = keras.initializers.get(initializer)

    def build(self, input_shape):
        output_dim = input_shape[-1]
        self.kernel = self.add_weight(
            shape=(output_dim * 2, output_dim),
            initializer=self.initializer,
            name="kernel",
            trainable=True,
        )

    def call(self, inputs):
        inputs -= tf.reduce_mean(inputs, axis=-1, keepdims=True)
        pos = tf.nn.relu(inputs)
        neg = tf.nn.relu(-inputs)
        concatenated = tf.concat([pos, neg], axis=-1)
        mixed = tf.matmul(concatenated, self.kernel)
        return mixed

    def get_config(self):
        # Implement get_config to enable serialization. This is optional.
        base_config = super(Antirectifier, self).get_config()
        config = {"initializer": keras.initializers.serialize(self.initializer)}
        return dict(list(base_config.items()) + list(config.items()))

time: 2.42 ms (started: 2021-08-26 23:00:43 +08:00)


让我们在 MNIST 上试驾一下

In [7]:
# Training parameters
batch_size = 128
num_classes = 10
epochs = 20

# The data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# Build the model
model = keras.Sequential(
    [
        keras.Input(shape=(784,)),
        layers.Dense(256),
        Antirectifier(),
        layers.Dense(256),
        Antirectifier(),
        layers.Dropout(0.5),
        layers.Dense(10),
    ]
)

# Compile the model
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.15)

# Test the model
model.evaluate(x_test, y_test)

60000 train samples
10000 test samples
Epoch 1/20
399/399 [==============================] - 6s 10ms/step - loss: 0.3845 - sparse_categorical_accuracy: 0.8872 - val_loss: 0.2007 - val_sparse_categorical_accuracy: 0.9458
Epoch 2/20
399/399 [==============================] - 4s 10ms/step - loss: 0.1762 - sparse_categorical_accuracy: 0.9511 - val_loss: 0.2024 - val_sparse_categorical_accuracy: 0.9469
Epoch 3/20
399/399 [==============================] - 4s 9ms/step - loss: 0.1402 - sparse_categorical_accuracy: 0.9623 - val_loss: 0.2007 - val_sparse_categorical_accuracy: 0.9542
Epoch 4/20
399/399 [==============================] - 4s 10ms/step - loss: 0.1160 - sparse_categorical_accuracy: 0.9685 - val_loss: 0.3924 - val_sparse_categorical_accuracy: 0.9252
Epoch 5/20
399/399 [==============================] - 4s 9ms/step - loss: 0.1038 - sparse_categorical_accuracy: 0.9733 - val_loss: 0.2550 - val_sparse_categorical_accuracy: 0.9466
Epoch 6/20
399/399 [==============================] - 4s 9

[0.21115773916244507, 0.9761000275611877]

time: 1min 23s (started: 2021-08-26 23:01:14 +08:00)


# 概率贝叶斯神经网络

## 介绍
对深度学习采用概率方法可以考虑不确定性，因此模型可以为不正确的预测分配较少的置信度。 由于标签或模型中的测量误差或噪声，由于模型有效学习的数据可用性不足，可以在数据中找到不确定性的来源。

这个例子演示了如何构建基本的概率贝叶斯神经网络来解释这两种类型的不确定性。 我们使用与 Keras API 兼容的 TensorFlow Probability 库。

此示例需要 TensorFlow 2.3 或更高版本。 您可以使用以下命令安装 Tensorflow Probability：

In [ ]:
pip install tensorflow-probability -U

## 数据集
我们使用 TensorFlow 数据集中提供的 Wine Quality 数据集。 我们使用 red wine 子集，其中包含 4,898 个示例。 该数据集具有葡萄酒的 11 个数值理化特征，任务是预测葡萄酒质量，即 0 到 10 之间的分数。在本示例中，我们将其视为回归任务。

您可以使用以下命令安装 TensorFlow 数据集：

In [ ]:
pip install tensorflow-datasets

## 设置

In [31]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

time: 978 µs (started: 2021-08-27 15:44:42 +08:00)


## 创建训练和评估数据集
在这里，我们使用 tfds.load() 加载 wine_quality 数据集，并将目标特征转换为浮点。 然后，我们打乱数据集并将其拆分为训练集和测试集。 我们将第一个 train_size 示例作为训练分割，其余作为测试分割。

In [32]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

time: 1.61 ms (started: 2021-08-27 15:44:45 +08:00)


## 编译、训练和评估模型

In [33]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

time: 916 µs (started: 2021-08-27 15:44:47 +08:00)


## 创建模型输入

In [34]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

time: 1.14 ms (started: 2021-08-27 15:44:49 +08:00)


## 实验一：标准神经网络
我们创建了一个标准的确定性神经网络模型作为基线。

In [35]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

time: 1.67 ms (started: 2021-08-27 15:44:51 +08:00)


让我们将 wine 数据集拆分为训练集和测试集，分别有 85% 和 15% 的示例

In [36]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

time: 130 ms (started: 2021-08-27 15:44:53 +08:00)


现在让我们训练基线模型。 我们使用 MeanSquaredError 作为损失函数。

In [16]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 2s 64ms/step - loss: 41.6719 - root_mean_squared_error: 6.4554 - val_loss: 38.9705 - val_root_mean_squared_error: 6.2426
Epoch 2/100
17/17 [==============================] - 0s 21ms/step - loss: 39.1777 - root_mean_squared_error: 6.2592 - val_loss: 37.3527 - val_root_mean_squared_error: 6.1117
Epoch 3/100
17/17 [==============================] - 0s 20ms/step - loss: 37.0985 - root_mean_squared_error: 6.0909 - val_loss: 35.6854 - val_root_mean_squared_error: 5.9737
Epoch 4/100
17/17 [==============================] - 0s 19ms/step - loss: 35.1338 - root_mean_squared_error: 5.9274 - val_loss: 33.9785 - val_root_mean_squared_error: 5.8291
Epoch 5/100
17/17 [==============================] - 0s 20ms/step - loss: 33.2495 - root_mean_squared_error: 5.7662 - val_loss: 32.2820 - val_root_mean_squared_error: 5.6817
Epoch 6/100
17/17 [==============================] - 0s 19ms/step - loss: 31.4391 - root_mean_squared_

我们从测试集中抽取样本使用该模型来获得对它们的预测。 请注意，由于基线模型是确定性的，因此我们为每个测试示例得到一个单点估计预测，没有关于模型和预测的不确定性的信息。

In [20]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

NameError: name 'baseline_model' is not defined

time: 145 ms (started: 2021-08-27 15:16:26 +08:00)


## 实验二：贝叶斯神经网络（BNN）
用于建模神经网络的贝叶斯方法的目标是捕获认知不确定性，即由于训练数据有限而导致的模型适应度的不确定性。

这个想法是，贝叶斯方法不是在神经网络中学习特定的权重（和偏差）值，而是学习权重分布——我们可以从中采样以产生给定输入的输出——以编码权重不确定性。

因此，我们需要定义这些权重的先验分布和后验分布，训练过程就是学习这些分布的参数。

In [8]:
# 将先验权重分布定义为均值=0 和标准偏差=1 的正态分布。 请注意，在这个例子中，
# 我们先验分布是不可训练的，因为我们固定了它的参数。

def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# 将变分后验权重分布定义为多元高斯分布。
# 请注意，此分布的可学习参数是均值、方差和协方差。
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

time: 1.57 ms (started: 2021-08-27 14:39:33 +08:00)


我们在神经网络模型中使用 `tfp.layers.DenseVariational` 层而不是标准的 `keras.layers.Dense` 层。

In [9]:
def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # 使用 DenseVariational 层创建权重不确定的隐藏层。
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # 输出是确定性的：单点估计。
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

time: 1.15 ms (started: 2021-08-27 14:41:22 +08:00)


随着我们增加训练数据的大小，可以减少认知不确定性。 也就是说，BNN 模型看到的数据越多，它对权重（分布参数）的估计就越确定。 让我们通过在训练集的一个小子集上训练 BNN 模型来测试这种行为，然后在整个训练集上比较输出方差。

### 用一个小的训练子集训练 BNN。

In [16]:
num_epochs = 500
mse_loss = keras.losses.MeanSquaredError()
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)

Start training the model...
Epoch 1/500
5/5 [==============================] - 4s 257ms/step - loss: 43.4080 - root_mean_squared_error: 6.5876 - val_loss: 43.5966 - val_root_mean_squared_error: 6.6019
Epoch 2/500
5/5 [==============================] - 0s 41ms/step - loss: 42.7788 - root_mean_squared_error: 6.5394 - val_loss: 41.4050 - val_root_mean_squared_error: 6.4335
Epoch 3/500
5/5 [==============================] - 0s 41ms/step - loss: 42.3316 - root_mean_squared_error: 6.5052 - val_loss: 38.6900 - val_root_mean_squared_error: 6.2187
Epoch 4/500
5/5 [==============================] - 0s 41ms/step - loss: 39.1338 - root_mean_squared_error: 6.2543 - val_loss: 37.6537 - val_root_mean_squared_error: 6.1350
Epoch 5/500
5/5 [==============================] - 0s 44ms/step - loss: 39.5230 - root_mean_squared_error: 6.2856 - val_loss: 38.1898 - val_root_mean_squared_error: 6.1783
Epoch 6/500
5/5 [==============================] - 0s 43ms/step - loss: 42.7919 - root_mean_squared_error: 6.54

由于我们已经训练了一个 BNN 模型，每次我们用相同的输入调用它时，该模型都会产生不同的输出，因为每次从分布中采样一组新的权重以构建网络并产生输出。 模式权重越不确定，我们将在相同输入的输出中看到的可变性越大（范围更广）。

In [21]:
def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 5.65, min: 5.0, max: 6.25, range: 1.25 - Actual: 6.0
Predictions mean: 5.5, min: 4.76, max: 6.02, range: 1.26 - Actual: 5.0
Predictions mean: 5.65, min: 4.84, max: 6.17, range: 1.33 - Actual: 5.0
Predictions mean: 5.51, min: 4.68, max: 6.13, range: 1.44 - Actual: 6.0
Predictions mean: 5.22, min: 4.38, max: 5.84, range: 1.46 - Actual: 5.0
Predictions mean: 5.75, min: 5.16, max: 6.17, range: 1.01 - Actual: 6.0
Predictions mean: 6.3, min: 5.93, max: 6.49, range: 0.56 - Actual: 6.0
Predictions mean: 6.11, min: 5.53, max: 6.39, range: 0.86 - Actual: 6.0
Predictions mean: 6.29, min: 5.7, max: 6.48, range: 0.77 - Actual: 8.0
Predictions mean: 5.56, min: 4.89, max: 6.08, range: 1.19 - Actual: 6.0
time: 8.9 s (started: 2021-08-27 15:16:56 +08:00)


### 用整个训练集训练 BNN

In [22]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)

compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 2s 48ms/step - loss: 38.9961 - root_mean_squared_error: 6.2444 - val_loss: 38.9452 - val_root_mean_squared_error: 6.2402
Epoch 2/500
17/17 [==============================] - 0s 18ms/step - loss: 40.6068 - root_mean_squared_error: 6.3720 - val_loss: 36.2418 - val_root_mean_squared_error: 6.0197
Epoch 3/500
17/17 [==============================] - 0s 18ms/step - loss: 37.4384 - root_mean_squared_error: 6.1183 - val_loss: 39.7948 - val_root_mean_squared_error: 6.3078
Epoch 4/500
17/17 [==============================] - 0s 22ms/step - loss: 34.4895 - root_mean_squared_error: 5.8724 - val_loss: 34.4035 - val_root_mean_squared_error: 5.8651
Epoch 5/500
17/17 [==============================] - 0s 18ms/step - loss: 34.5635 - root_mean_squared_error: 5.8787 - val_loss: 28.9765 - val_root_mean_squared_error: 5.3826
Epoch 6/500
17/17 [==============================] - 0s 18ms/step - loss: 33.8124 - root_mean_squared_

## 实验 3：概率贝叶斯神经网络
到目前为止，我们构建的标准和贝叶斯 NN 模型的输出是确定性的，即生成点估计作为给定示例的预测。 我们可以通过让模型输出分布来创建概率神经网络。 在这种情况下，该模型也捕获了随机不确定性，这是由于数据中不可约的噪声或生成数据过程的随机性。

在这个例子中，我们将输出建模为一个独立正态分布，具有可学习的均值和方差参数。 如果任务是分类，我们将使用带有二元类的 IndependentBernoulli 和带有多个类的 OneHotCategorical 来对模型输出的分布进行建模。

In [23]:
def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # 使用 DenseVariational 层创建具有权重不确定性的隐藏层。
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # 创建一个概率输出（正态分布），并使用 `Dense` 层
    # 产生分布的参数。
    # 我们设置units=2来学习正态分布的均值和方差。
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

time: 1.65 ms (started: 2021-08-27 15:26:03 +08:00)


由于模型的输出是分布，而不是点估计，我们使用负对数似然作为我们的损失函数来计算从模型产生的估计分布中看到真实数据（目标）的可能性。

In [24]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)

Start training the model...
Epoch 1/1000
17/17 [==============================] - 2s 47ms/step - loss: 94.2285 - root_mean_squared_error: 5.9486 - val_loss: 61.3006 - val_root_mean_squared_error: 5.6013
Epoch 2/1000
17/17 [==============================] - 0s 18ms/step - loss: 62.5013 - root_mean_squared_error: 5.8447 - val_loss: 44.0424 - val_root_mean_squared_error: 5.4398
Epoch 3/1000
17/17 [==============================] - 0s 18ms/step - loss: 58.8646 - root_mean_squared_error: 5.6543 - val_loss: 90.6804 - val_root_mean_squared_error: 5.9676
Epoch 4/1000
17/17 [==============================] - 0s 20ms/step - loss: 43.2510 - root_mean_squared_error: 5.7106 - val_loss: 28.1915 - val_root_mean_squared_error: 5.6210
Epoch 5/1000
17/17 [==============================] - 0s 21ms/step - loss: 37.2680 - root_mean_squared_error: 5.5712 - val_loss: 27.4590 - val_root_mean_squared_error: 5.4211
Epoch 6/1000
17/17 [==============================] - 0s 18ms/step - loss: 36.4679 - root_mean_sq

现在让我们从给定测试示例的模型中生成输出。 输出现在是一个分布，我们可以使用它的均值和方差来计算预测的置信区间 (CI)。

In [37]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 5.56, stddev: 0.74, 95% CI: [7.01 - 4.1] - Actual: 6.0
Prediction mean: 5.03, stddev: 0.65, 95% CI: [6.31 - 3.75] - Actual: 5.0
Prediction mean: 5.16, stddev: 0.68, 95% CI: [6.49 - 3.83] - Actual: 5.0
Prediction mean: 5.12, stddev: 0.67, 95% CI: [6.44 - 3.8] - Actual: 6.0
Prediction mean: 4.93, stddev: 0.64, 95% CI: [6.2 - 3.67] - Actual: 5.0
Prediction mean: 5.94, stddev: 0.79, 95% CI: [7.49 - 4.39] - Actual: 6.0
Prediction mean: 6.5, stddev: 0.84, 95% CI: [8.14 - 4.86] - Actual: 6.0
Prediction mean: 6.14, stddev: 0.8, 95% CI: [7.71 - 4.58] - Actual: 6.0
Prediction mean: 6.54, stddev: 0.84, 95% CI: [8.18 - 4.9] - Actual: 8.0
Prediction mean: 5.47, stddev: 0.73, 95% CI: [6.9 - 4.03] - Actual: 6.0
time: 195 ms (started: 2021-08-27 15:45:25 +08:00)


# 知识蒸馏配方

## 介绍
知识蒸馏（Hinton 等人）是一种使我们能够将较大模型压缩为较小模型的技术。 这使我们能够获得高性能大型模型的好处，同时降低存储和内存成本并实现更高的推理速度：
* 更小的模型 -> 更小的内存占用
* 降低复杂性 -> 更少的浮点运算 (FLOP)

在知识蒸馏中：一个好老师是耐心和一致的，Beyer 等人。研究用于执行知识蒸馏的各种现有设置，并表明所有这些设置都会导致次优性能。因此，在开发资源受限的生产系统时，从业者通常会接受其他替代方案（量化、修剪、权重聚类等）。

Beyer 等人研究了我们如何改进知识提炼过程中产生的学生模型，并始终与他们的教师模型的表现相匹配。在这个例子中，我们将使用 Flowers102 数据集研究他们介绍的食谱。作为参考，通过这些方法，作者能够生成一个 ResNet50 模型，该模型在 ImageNet-1k 数据集上实现了 82.8% 的准确率。

如果你需要复习知识提炼，并想学习它是如何在 Keras 中实现的，你可以参考这个例子。您还可以按照此示例显示应用于一致性训练的知识蒸馏扩展。

要遵循此示例，您将需要 TensorFlow 2.5 或更高版本以及 TensorFlow Addons，可以使用以下命令进行安装：

In [ ]:
pip install tensorflow-addons -U

## 倒入库

In [39]:
from tensorflow import keras
import tensorflow_addons as tfa
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

import tensorflow_datasets as tfds

tfds.disable_progress_bar()

time: 340 ms (started: 2021-08-27 15:51:53 +08:00)


## 超参数和内容

In [42]:
AUTO = tf.data.AUTOTUNE  # 用于动态调整并行度。
BATCH_SIZE = 64 #来自表 4 和“训练设置”部分。
# 用于在 logits 进入 softmax 之前软化它们。
# 来自表 4 和“训练设置”部分。
TEMPERATURE = 10  # 用于在 logits 进入 softmax 之前软化它们。
INIT_LR = 0.003  # 将在训练期间衰减的初始学习率。
WEIGHT_DECAY = 0.001  # 用于正则化。
CLIP_THRESHOLD = 1.0  # 用于通过 L2 范数裁剪梯度。

# 我们将首先将训练图像调整为更大的尺寸，然后我们将采用较小尺寸的随机裁剪。
BIGGER = 160
RESIZE = 128

time: 1.11 ms (started: 2021-08-27 15:54:22 +08:00)


## 加载 Flowers102 数据集

In [ ]:
train_ds, validation_ds, test_ds = tfds.load(
    "oxford_flowers102", split=["train", "validation", "test"], as_supervised=True
)

print(f"Number of training examples: {tf.data.experimental.cardinality(train_ds)}.")

print(
    f"Number of validation examples: {tf.data.experimental.cardinality(validation_ds)}."
)

print(f"Number of test examples: {tf.data.experimental.cardinality(test_ds)}.")

## 教师模式
与任何蒸馏技术一样，重要的是首先训练一个表现良好的教师模型，该模型通常比随后的学生模型大。 作者将一个 BiT ResNet152x2 模型（教师）提炼成一个 BiT ResNet50 模型（学生）。

BiT 代表 Big Transfer，是在 Big Transfer (BiT): General Visual Representation Learning 中引入的。 ResNets 的 BiT 变体使用 Group Normalization（Wu 等人）和 Weight Standardization（Qiao 等人）代替 Batch Normalization（Ioffe 等人）。 为了限制运行此示例所需的时间，我们将使用已经在 Flowers102 数据集上训练的 BiT ResNet101x3。 您可以参考此笔记本以了解有关培训过程的更多信息。 该模型在 Flowers102 的测试集上达到了 98.18% 的准确率。

模型权重作为数据集托管在 Kaggle 上。 要下载权重，请按照下列步骤操作：
1. 在此处在 Kaggle 上创建一个帐户。
2. 转到用户个人资料的“帐户”选项卡。
3. 选择“创建 API 令牌”。 这将触发 kaggle.json 的下载，该文件包含您的 API 凭据。
4. 从该 JSON 文件中，复制您的 Kaggle 用户名和 API 密钥。

现在运行以下命令：

In [ ]:
import os

os.environ["KAGGLE_USERNAME"] = "" # TODO: enter your Kaggle user name here
os.environ["KAGGLE_KEY"] = "" # TODO: enter your Kaggle key here

# 使用 TensorFlow NumPy 编写 Keras 模型

## 介绍

NumPy 是一个非常成功的 Python 线性代数库。

TensorFlow 最近推出了 `tf_numpy`，这是 NumPy API 的一个大型子集的 TensorFlow 实现。 感谢 `tf_numpy`，您可以以 NumPy 风格编写 Keras 层或模型！

TensorFlow NumPy API 与 TensorFlow 生态系统完全集成。 支持自动微分、TensorBoard、Keras 模型回调、TPU 分发和模型导出等功能。

让我们来看看几个例子。

## 设置
TensorFlow NumPy 需要 TensorFlow 2.5 或更高版本。

In [44]:
import tensorflow as tf
import tensorflow.experimental.numpy as tnp
import keras
import keras.layers as layers
import numpy as np

time: 23.8 ms (started: 2021-08-27 16:10:23 +08:00)


或者，您可以调用 `tnp.experimental_enable_numpy_behavior()` 以在 TensorFlow 中启用类型提升。 这允许 TNP 更紧密地遵循 NumPy 标准。

In [45]:
tnp.experimental_enable_numpy_behavior()

time: 2.14 ms (started: 2021-08-27 16:23:21 +08:00)


为了测试我们的模型，我们将使用波士顿房价回归数据集。

In [46]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def evaluate_model(model: keras.Model):
    [loss, percent_error] = model.evaluate(x_test, y_test, verbose=0)
    print("Mean absolute percent error before training: ", percent_error)
    model.fit(x_train, y_train, epochs=200, verbose=0)
    [loss, percent_error] = model.evaluate(x_test, y_test, verbose=0)
    print("Mean absolute percent error after training:", percent_error)

time: 30.7 ms (started: 2021-08-27 16:24:47 +08:00)


## 使用 TNP 对 keras.Model 进行子类化
使用 Keras API 最灵活的方法是将 [keras.Model](/api/models/model#model-class) 类子类化。 对模型类进行子类化使您能够完全自定义训练循环中发生的事情。 这使得子类模型成为研究人员的流行选择。

在这个例子中，我们将实现一个模型子类，它使用 TNP API 对波士顿住房数据集执行回归。 请注意，将 TNP API 与 keras 一起使用时，会自动处理微分和梯度下降。

首先让我们定义一个简单的 `TNPForwardFeedRegressionNetwork` 类。

In [47]:
class TNPForwardFeedRegressionNetwork(keras.Model):
    def __init__(self, blocks=None, **kwargs):
        super(TNPForwardFeedRegressionNetwork, self).__init__(**kwargs)
        if not isinstance(blocks, list):
            raise ValueError(f"blocks must be a list, got blocks={blocks}")
        self.blocks = blocks
        self.block_weights = None
        self.biases = None

    def build(self, input_shape):
        current_shape = input_shape[1]
        self.block_weights = []
        self.biases = []
        for i, block in enumerate(self.blocks):
            self.block_weights.append(
                self.add_weight(
                    shape=(current_shape, block), trainable=True, name=f"block-{i}"
                )
            )
            self.biases.append(
                self.add_weight(shape=(block,), trainable=True, name=f"bias-{i}")
            )
            current_shape = block

        self.linear_layer = self.add_weight(
            shape=(current_shape, 1), name="linear_projector", trainable=True
        )

    def call(self, inputs):
        activations = inputs
        for w, b in zip(self.block_weights, self.biases):
            activations = tnp.matmul(activations, w) + b
            # ReLu activation function
            activations = tnp.maximum(activations, 0.0)

        return tnp.matmul(activations, self.linear_layer)

time: 1.51 ms (started: 2021-08-27 17:00:33 +08:00)


就像任何其他 Keras 模型一样，我们可以利用我们想要的任何受支持的优化器、损失、指标或回调。

让我们看看模型的表现如何！

In [48]:
model = TNPForwardFeedRegressionNetwork(blocks=[3, 3])
model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[keras.metrics.MeanAbsolutePercentageError()],
)
evaluate_model(model)

Mean absolute percent error before training:  1863.6688232421875
Mean absolute percent error after training: 29.483413696289062
time: 22.8 s (started: 2021-08-27 17:14:30 +08:00)


伟大的！ 我们的模型似乎正在有效地学习解决手头的问题。

我们还可以使用 TNP 编写我们自己的自定义损失函数。

In [49]:
def tnp_mse(y_true, y_pred):
    return tnp.mean(tnp.square(y_true - y_pred), axis=0)


keras.backend.clear_session()
model = TNPForwardFeedRegressionNetwork(blocks=[3, 3])
model.compile(
    optimizer="adam",
    loss=tnp_mse,
    metrics=[keras.metrics.MeanAbsolutePercentageError()],
)
evaluate_model(model)

Mean absolute percent error before training:  799.2676391601562
Mean absolute percent error after training: 32.759307861328125
time: 22.8 s (started: 2021-08-27 17:15:08 +08:00)


## 使用 TNP 实现基于 Keras 层的模型
如果需要，TNP 也可以用于面向层的 Keras 代码结构。 让我们实现相同的模型，但使用分层方法！

In [50]:
def tnp_relu(x):
    return tnp.maximum(x, 0)


class TNPDense(keras.layers.Layer):
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        self.w = self.add_weight(
            name="weights",
            shape=(input_shape[1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.bias = self.add_weight(
            name="bias",
            shape=(self.units,),
            initializer="random_normal",
            trainable=True,
        )

    def call(self, inputs):
        outputs = tnp.matmul(inputs, self.w) + self.bias
        if self.activation:
            return self.activation(outputs)
        return outputs

def create_layered_tnp_model():
    return keras.Sequential(
        [
            TNPDense(3, activation=tnp_relu),
            TNPDense(3, activation=tnp_relu),
            TNPDense(1),
        ]
    )


model = create_layered_tnp_model()
model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[keras.metrics.MeanAbsolutePercentageError()],
)
model.build((None, 13,))
model.summary()

evaluate_model(model)    

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tnp_dense (TNPDense)         (None, 3)                 42        
_________________________________________________________________
tnp_dense_1 (TNPDense)       (None, 3)                 12        
_________________________________________________________________
tnp_dense_2 (TNPDense)       (None, 1)                 4         
Total params: 58
Trainable params: 58
Non-trainable params: 0
_________________________________________________________________
Mean absolute percent error before training:  99.54196166992188
Mean absolute percent error after training: 21.16244888305664
time: 23.4 s (started: 2021-08-27 17:22:39 +08:00)


您还可以在 TNP 层和原生 Keras 层之间无缝切换！

In [51]:
def create_mixed_model():
    return keras.Sequential(
        [
            TNPDense(3, activation=tnp_relu),
            # The model will have no issue using a normal Dense layer
            layers.Dense(3, activation="relu"),
            # ... or switching back to tnp layers!
            TNPDense(1),
        ]
    )


model = create_mixed_model()
model.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics=[keras.metrics.MeanAbsolutePercentageError()],
)
model.build((None, 13,))
model.summary()

evaluate_model(model)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tnp_dense_3 (TNPDense)       (None, 3)                 42        
_________________________________________________________________
dense (Dense)                (None, 3)                 12        
_________________________________________________________________
tnp_dense_4 (TNPDense)       (None, 1)                 4         
Total params: 58
Trainable params: 58
Non-trainable params: 0
_________________________________________________________________
Mean absolute percent error before training:  103.97216033935547
Mean absolute percent error after training: 28.8846435546875
time: 24.1 s (started: 2021-08-27 17:27:35 +08:00)


Keras API 提供了各种各样的层。 将它们与 NumPy 代码一起使用的能力可以在项目中节省大量时间。

## Distribution方案

TensorFlow NumPy 和 Keras 与 TensorFlow Distribution Strategies 集成。 这使得跨多个 GPU 甚至整个 TPU Pod 执行分布式训练变得简单。

In [52]:
gpus = tf.config.list_logical_devices("GPU")
if gpus:
    strategy = tf.distribute.MirroredStrategy(gpus)
else:
    # 我们可以回退到无操作 CPU 策略。
    strategy = tf.distribute.get_strategy()
print("Running with strategy:", str(strategy.__class__.__name__))

with strategy.scope():
    model = create_layered_tnp_model()
    model.compile(
        optimizer="adam",
        loss="mean_squared_error",
        metrics=[keras.metrics.MeanAbsolutePercentageError()],
    )
    model.build((None, 13,))
    model.summary()
    evaluate_model(model)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


Running with strategy: MirroredStrategy
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tnp_dense_5 (TNPDense)       (None, 3)                 42        
_________________________________________________________________
tnp_dense_6 (TNPDense)       (None, 3)                 12        
_________________________________________________________________
tnp_dense_7 (TNPDense)       (None, 1)                 4         
Total params: 58
Trainable params: 58
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Mean absolute percent error before training:  100.1906509399414
Mean absolute percent error after training: 22.98281478881836
time: 41.8 s (started: 2021-08-27 17:34:19 +08:00)


## TensorBoard 集成
使用 Keras API 的众多好处之一是能够通过 TensorBoard 监控训练。 将 TensorFlow NumPy API 与 Keras 结合使用，您可以轻松利用 TensorBoard。

In [53]:
keras.backend.clear_session()

time: 35.5 ms (started: 2021-08-27 17:36:34 +08:00)


要从 Jupyter 笔记本加载 TensorBoard，您可以运行以下魔术：

In [54]:
%load_ext tensorboard

time: 13.9 ms (started: 2021-08-27 17:37:41 +08:00)


In [55]:
models = [
    (TNPForwardFeedRegressionNetwork(blocks=[3, 3]), "TNPForwardFeedRegressionNetwork"),
    (create_layered_tnp_model(), "layered_tnp_model"),
    (create_mixed_model(), "mixed_model"),
]
for model, model_name in models:
    model.compile(
        optimizer="adam",
        loss="mean_squared_error",
        metrics=[keras.metrics.MeanAbsolutePercentageError()],
    )
    model.fit(
        x_train,
        y_train,
        epochs=200,
        verbose=0,
        callbacks=[keras.callbacks.TensorBoard(log_dir=f"logs/{model_name}")],
    )

time: 1min 16s (started: 2021-08-27 17:37:59 +08:00)


要从 Jupyter 笔记本加载 TensorBoard，您可以使用`%tensorboard` 魔法：

In [58]:
!kill 115110

time: 212 ms (started: 2021-08-27 17:42:48 +08:00)


In [ ]:
%reload_ext tensorboard
#%tensorboard --logdir ./data/tensorboard

from tensorboard import notebook
notebook.list() 

#在tensorboard中查看模型
notebook.start("--logdir logs")

TensorBoard 监控指标并检查训练曲线。

TensorBoard 还允许您探索模型中使用的计算图。

内省模型的能力在调试过程中很有价值。

## 结论
使用 tensorflow_numpy API 将现有的 NumPy 代码移植到 Keras 模型很容易！ 通过与 Keras 集成，您可以获得使用现有 Keras 回调、指标和优化器的能力，轻松分发您的训练并使用 Tensorboard。

将更复杂的模型（例如 ResNet）迁移到 TensorFlow NumPy API 将是一个很好的后续学习练习。

在线提供了几个开源 NumPy ResNet 实现。